# Memory
* Save your conversation with the LLM.

## Intro
* Ability to store information about past interactions.
* **Most of memory-related functionality in LangChain is marked as beta**. This is for two reasons:
    1. Most functionality are not production ready.
    2. Most functionality work with Legacy chains, not the newer LCEL syntax.
* **The main exception to this is the ChatMessageHistory functionality**. This functionality is largely production ready and does integrate with LCEL.

## LangChain documentation on Memory
* See the LangChain documentation page on Memory [here](https://python.langchain.com/v0.1/docs/modules/memory/).
* See the LangChain documentation page on how to use ChatMessageHistory with LCEL [here](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).
* See the LangChain documentation page on the various ChatMessageHistory integrations [here](https://python.langchain.com/v0.1/docs/integrations/memory/).

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-simple-chain.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-conversation-buffer**.

## Track operations
From now on, we can track the operations **and the cost** of this project from LangSmith:
* [smith.langchain.com](https://smith.langchain.com)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [1]:
#pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

## Buffer Memory
* ConversationBufferMemory keeps a list of chat messages in a buffer and passes those into the prompt template.
* A buffer refers to a **temporary storage area in memory** used to hold data. Here, the buffer specifically holds chat messages before they are processed or used in some way, such as being passed into a prompt template.
* In simple terms, a buffer is like a waiting room for data. It's a temporary holding spot where data can stay until it's ready to be used or processed. Imagine you're cooking and you chop up vegetables before you need to cook them. Instead of chopping each vegetable right before it goes in the pan, you chop them all at once and put them aside on a cutting board. That cutting board with the chopped vegetables is like a buffer — it holds everything ready for you until you're ready to use it. This way, when you need the vegetables, they are all prepared and ready to go, making the cooking process smoother and faster.
* As you can see in the following example, ConversationBufferMemory was used mostly in the initial versions of LangChain.

In [6]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

/var/folders/r4/zk4dfhln1l78pwrms6p0867h0000gn/T/ipykernel_1997/1371371747.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/var/folders/r4/zk4dfhln1l78pwrms6p0867h0000gn/T/ipykernel_1997/1371371747.py:25: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


In [7]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})

/var/folders/r4/zk4dfhln1l78pwrms6p0867h0000gn/T/ipykernel_1997/1120294461.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  conversation({"question": "hi"})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'text': 'Hello! How can I assist you today?'}

In [8]:
conversation({"question": "My name is Beto and I have moved 33 times."})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How can I assist you today?
Human: My name is Beto and I have moved 33 times.

> Finished chain.


{'question': 'My name is Beto and I have moved 33 times.',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Beto and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?', additional_kwargs={}, response_metadata={})],
 'text': 'Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?'}

In [9]:
conversation({"question": "If my average moving distance was 100 miles, how many miles took all my moves?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How can I assist you today?
Human: My name is Beto and I have moved 33 times.
AI: Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?
Human: If my average moving distance was 100 miles, how many miles took all my moves?

> Finished chain.


{'question': 'If my average moving distance was 100 miles, how many miles took all my moves?',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Beto and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="If you've moved 33 times with an average distance of 100 miles each time, you can calculate the total distance by multiplying the number of moves by the average distance:\n\n33 moves × 100 miles/move = 3,300 miles\n\nSo, you’ve moved a total of 3,300 miles! That's

In [10]:
conversation({"question": "Do you remember my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How can I assist you today?
Human: My name is Beto and I have moved 33 times.
AI: Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?
Human: If my average moving distance was 100 miles, how many miles took all my moves?
AI: If you've moved 33 times with an average distance of 100 miles each time, you can calculate the total distance by multiplying the number of moves by the average distance:

33 moves × 100 miles/move = 3,300 miles

So, you’ve moved a total of 3,300 miles! That's quite a journey!
Human: Do you remember my name?

> Finished chain.


{'question': 'Do you remember my name?',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Beto and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="If you've moved 33 times with an average distance of 100 miles each time, you can calculate the total distance by multiplying the number of moves by the average distance:\n\n33 moves × 100 miles/move = 3,300 miles\n\nSo, you’ve moved a total of 3,300 miles! That's quite a journey!", additional_kwargs={}, response_met

In [11]:
print(memory.buffer)

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='My name is Beto and I have moved 33 times.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?', additional_kwargs={}, response_metadata={}), HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}), AIMessage(content="If you've moved 33 times with an average distance of 100 miles each time, you can calculate the total distance by multiplying the number of moves by the average distance:\n\n33 moves × 100 miles/move = 3,300 miles\n\nSo, you’ve moved a total of 3,300 miles! That's quite a journey!", additional_kwargs={}, response_metadata={}), HumanMessage(content='Do you remember my name?', addition

In [12]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Beto and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="If you've moved 33 times with an average distance of 100 miles each time, you can calculate the total distance by multiplying the number of moves by the average distance:\n\n33 moves × 100 miles/move = 3,300 miles\n\nSo, you’ve moved a total of 3,300 miles! That's quite a journey!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Do you

In [13]:
conversation({"question": "The most interesting place I've lived in is Vancouver. I can't wait to return to that amazing place"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How can I assist you today?
Human: My name is Beto and I have moved 33 times.
AI: Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?
Human: If my average moving distance was 100 miles, how many miles took all my moves?
AI: If you've moved 33 times with an average distance of 100 miles each time, you can calculate the total distance by multiplying the number of moves by the average distance:

33 moves × 100 miles/move = 3,300 miles

So, you’ve moved a total of 3,300 miles! That's quite a journey!
Human: Do you remember my name?
AI: Yes, your name is Beto! How can I help you further today?
Human: The most interesting place I've lived in is Vancouver. I can't wait to return to that amazing place

> Finished chain.


{'question': "The most interesting place I've lived in is Vancouver. I can't wait to return to that amazing place",
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Beto and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Beto! That’s quite a lot of moving! What’s been the most interesting place you’ve lived in?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="If you've moved 33 times with an average distance of 100 miles each time, you can calculate the total distance by multiplying the number of moves by the average distance:\n\n33 moves × 100 miles/move = 3,300 miles\n\nSo, you’ve moved a total o

## Conversation Buffer Window Memory
Similar to the previous one, but **you can limit the number of conversational exchanges stored in memory**. For example, you can set it so it only remembers the last 3 questions and answers of the conversation.

In [14]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [15]:
window_memory = ConversationBufferWindowMemory(k=3)

/var/folders/r4/zk4dfhln1l78pwrms6p0867h0000gn/T/ipykernel_1997/2492508394.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory = ConversationBufferWindowMemory(k=3)


In [16]:
conversation_window = ConversationChain(
    llm=llm, 
    memory = window_memory,
    verbose=True
)

/var/folders/r4/zk4dfhln1l78pwrms6p0867h0000gn/T/ipykernel_1997/2841821209.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_window = ConversationChain(


In [17]:
conversation_window({"input": "Hi, my name is Beto"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Beto
AI:

> Finished chain.


{'input': 'Hi, my name is Beto',
 'history': '',
 'response': "Hello Beto! It's great to meet you! How's your day going so far?"}

In [18]:
conversation_window({"input": "My favorite color is black"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Beto
AI: Hello Beto! It's great to meet you! How's your day going so far?
Human: My favorite color is black
AI:

> Finished chain.


{'input': 'My favorite color is black',
 'history': "Human: Hi, my name is Beto\nAI: Hello Beto! It's great to meet you! How's your day going so far?",
 'response': "Black is such a classic choice—it's elegant and can be quite versatile! Do you like wearing black, or do you prefer it in your decor? It can really set a mood, either way!"}

In [19]:
conversation_window({"input": "My favorite animals are dogs"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Beto
AI: Hello Beto! It's great to meet you! How's your day going so far?
Human: My favorite color is black
AI: Black is such a classic choice—it's elegant and can be quite versatile! Do you like wearing black, or do you prefer it in your decor? It can really set a mood, either way!
Human: My favorite animals are dogs
AI:

> Finished chain.


{'input': 'My favorite animals are dogs',
 'history': "Human: Hi, my name is Beto\nAI: Hello Beto! It's great to meet you! How's your day going so far?\nHuman: My favorite color is black\nAI: Black is such a classic choice—it's elegant and can be quite versatile! Do you like wearing black, or do you prefer it in your decor? It can really set a mood, either way!",
 'response': 'Dogs are wonderful companions! They’re known for their loyalty and playful nature. Do you have a dog, or is there a particular breed you love the most? There are so many types, from the spirited Beagle to the gentle Great Dane!'}

In [20]:
conversation_window({"input": "I'd like to drive a red Ferrari in the city"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Beto
AI: Hello Beto! It's great to meet you! How's your day going so far?
Human: My favorite color is black
AI: Black is such a classic choice—it's elegant and can be quite versatile! Do you like wearing black, or do you prefer it in your decor? It can really set a mood, either way!
Human: My favorite animals are dogs
AI: Dogs are wonderful companions! They’re known for their loyalty and playful nature. Do you have a dog, or is there a particular breed you love the most? There are so many types, from the spirited Beagle to the gentle Great Dane!
Human: I'd like to drive a red Ferrari in the city
AI:

> Finished chain.


{'input': "I'd like to drive a red Ferrari in the city",
 'history': "Human: Hi, my name is Beto\nAI: Hello Beto! It's great to meet you! How's your day going so far?\nHuman: My favorite color is black\nAI: Black is such a classic choice—it's elegant and can be quite versatile! Do you like wearing black, or do you prefer it in your decor? It can really set a mood, either way!\nHuman: My favorite animals are dogs\nAI: Dogs are wonderful companions! They’re known for their loyalty and playful nature. Do you have a dog, or is there a particular breed you love the most? There are so many types, from the spirited Beagle to the gentle Great Dane!",
 'response': 'Driving a red Ferrari through the city sounds exhilarating! Ferraris are renowned for their speed, sleek designs, and that iconic roar of the engine. The vibrant red color adds an extra flair, making it a real head-turner on the streets. Do you have a favorite model in mind? The Ferrari 488 GTB is a fantastic choice with its powerful

In [21]:
conversation_window({"input": "My favorite city is Vancouver"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite color is black
AI: Black is such a classic choice—it's elegant and can be quite versatile! Do you like wearing black, or do you prefer it in your decor? It can really set a mood, either way!
Human: My favorite animals are dogs
AI: Dogs are wonderful companions! They’re known for their loyalty and playful nature. Do you have a dog, or is there a particular breed you love the most? There are so many types, from the spirited Beagle to the gentle Great Dane!
Human: I'd like to drive a red Ferrari in the city
AI: Driving a red Ferrari through the city sounds exhilarating! Ferraris are renowned for their speed, sleek designs, and that iconic roar of the en

{'input': 'My favorite city is Vancouver',
 'history': "Human: My favorite color is black\nAI: Black is such a classic choice—it's elegant and can be quite versatile! Do you like wearing black, or do you prefer it in your decor? It can really set a mood, either way!\nHuman: My favorite animals are dogs\nAI: Dogs are wonderful companions! They’re known for their loyalty and playful nature. Do you have a dog, or is there a particular breed you love the most? There are so many types, from the spirited Beagle to the gentle Great Dane!\nHuman: I'd like to drive a red Ferrari in the city\nAI: Driving a red Ferrari through the city sounds exhilarating! Ferraris are renowned for their speed, sleek designs, and that iconic roar of the engine. The vibrant red color adds an extra flair, making it a real head-turner on the streets. Do you have a favorite model in mind? The Ferrari 488 GTB is a fantastic choice with its powerful V8 engine and stunning looks, but the classic Ferrari F40 is a true ic

In [22]:
conversation_window({"input": "My favorite season is summer"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite animals are dogs
AI: Dogs are wonderful companions! They’re known for their loyalty and playful nature. Do you have a dog, or is there a particular breed you love the most? There are so many types, from the spirited Beagle to the gentle Great Dane!
Human: I'd like to drive a red Ferrari in the city
AI: Driving a red Ferrari through the city sounds exhilarating! Ferraris are renowned for their speed, sleek designs, and that iconic roar of the engine. The vibrant red color adds an extra flair, making it a real head-turner on the streets. Do you have a favorite model in mind? The Ferrari 488 GTB is a fantastic choice with its powerful V8 engine and stun

{'input': 'My favorite season is summer',
 'history': "Human: My favorite animals are dogs\nAI: Dogs are wonderful companions! They’re known for their loyalty and playful nature. Do you have a dog, or is there a particular breed you love the most? There are so many types, from the spirited Beagle to the gentle Great Dane!\nHuman: I'd like to drive a red Ferrari in the city\nAI: Driving a red Ferrari through the city sounds exhilarating! Ferraris are renowned for their speed, sleek designs, and that iconic roar of the engine. The vibrant red color adds an extra flair, making it a real head-turner on the streets. Do you have a favorite model in mind? The Ferrari 488 GTB is a fantastic choice with its powerful V8 engine and stunning looks, but the classic Ferrari F40 is a true icon too! What do you imagine doing while driving it around the city?\nHuman: My favorite city is Vancouver\nAI: Vancouver is such a beautiful city! It's surrounded by stunning natural scenery with mountains, ocean,

In [23]:
conversation_window({"input": "What is my favorite color?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'd like to drive a red Ferrari in the city
AI: Driving a red Ferrari through the city sounds exhilarating! Ferraris are renowned for their speed, sleek designs, and that iconic roar of the engine. The vibrant red color adds an extra flair, making it a real head-turner on the streets. Do you have a favorite model in mind? The Ferrari 488 GTB is a fantastic choice with its powerful V8 engine and stunning looks, but the classic Ferrari F40 is a true icon too! What do you imagine doing while driving it around the city?
Human: My favorite city is Vancouver
AI: Vancouver is such a beautiful city! It's surrounded by stunning natural scenery with mountains, ocean, and 

{'input': 'What is my favorite color?',
 'history': "Human: I'd like to drive a red Ferrari in the city\nAI: Driving a red Ferrari through the city sounds exhilarating! Ferraris are renowned for their speed, sleek designs, and that iconic roar of the engine. The vibrant red color adds an extra flair, making it a real head-turner on the streets. Do you have a favorite model in mind? The Ferrari 488 GTB is a fantastic choice with its powerful V8 engine and stunning looks, but the classic Ferrari F40 is a true icon too! What do you imagine doing while driving it around the city?\nHuman: My favorite city is Vancouver\nAI: Vancouver is such a beautiful city! It's surrounded by stunning natural scenery with mountains, ocean, and lush green parks. The combination of urban life and nature is just incredible! You can stroll along the seawall, explore Stanley Park, or check out Granville Island's vibrant market. Do you have a favorite spot in Vancouver or a specific activity you enjoy doing ther

In [24]:
conversation_window({"input": "My favorite city is Vancouver, but I also like Paris"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite city is Vancouver
AI: Vancouver is such a beautiful city! It's surrounded by stunning natural scenery with mountains, ocean, and lush green parks. The combination of urban life and nature is just incredible! You can stroll along the seawall, explore Stanley Park, or check out Granville Island's vibrant market. Do you have a favorite spot in Vancouver or a specific activity you enjoy doing there, like hiking, dining, or attending events? The city also has a diverse culinary scene with influences from all over the world!
Human: My favorite season is summer
AI: Summer is such a delightful season! It's the time for warm weather, longer days, and outdoor 

{'input': 'My favorite city is Vancouver, but I also like Paris',
 'history': "Human: My favorite city is Vancouver\nAI: Vancouver is such a beautiful city! It's surrounded by stunning natural scenery with mountains, ocean, and lush green parks. The combination of urban life and nature is just incredible! You can stroll along the seawall, explore Stanley Park, or check out Granville Island's vibrant market. Do you have a favorite spot in Vancouver or a specific activity you enjoy doing there, like hiking, dining, or attending events? The city also has a diverse culinary scene with influences from all over the world!\nHuman: My favorite season is summer\nAI: Summer is such a delightful season! It's the time for warm weather, longer days, and outdoor adventures. Whether it's going to the beach, enjoying a picnic in the park, or taking part in summer festivals, there’s so much to love. Plus, summer is great for fresh fruits and vegetables, like juicy watermelons, ripe peaches, and vibrant

In [25]:
conversation_window({"input": "Actually, I do. Le COnsulat is my favorite restaurant in Paris. I love their food!. I Also like the Louvre Museum."})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite season is summer
AI: Summer is such a delightful season! It's the time for warm weather, longer days, and outdoor adventures. Whether it's going to the beach, enjoying a picnic in the park, or taking part in summer festivals, there’s so much to love. Plus, summer is great for fresh fruits and vegetables, like juicy watermelons, ripe peaches, and vibrant tomatoes! Do you have any favorite summer activities or traditions? Perhaps you enjoy traveling, hiking, or simply soaking up the sun?
Human: What is my favorite color?
AI: I don't know your favorite color! But if you want to share it, I'd love to hear about it. Colors can have such interesting meanin

{'input': 'Actually, I do. Le COnsulat is my favorite restaurant in Paris. I love their food!. I Also like the Louvre Museum.',
 'history': "Human: My favorite season is summer\nAI: Summer is such a delightful season! It's the time for warm weather, longer days, and outdoor adventures. Whether it's going to the beach, enjoying a picnic in the park, or taking part in summer festivals, there’s so much to love. Plus, summer is great for fresh fruits and vegetables, like juicy watermelons, ripe peaches, and vibrant tomatoes! Do you have any favorite summer activities or traditions? Perhaps you enjoy traveling, hiking, or simply soaking up the sun?\nHuman: What is my favorite color?\nAI: I don't know your favorite color! But if you want to share it, I'd love to hear about it. Colors can have such interesting meanings and effects on our moods. Do you have a particular color that you find yourself drawn to?\nHuman: My favorite city is Vancouver, but I also like Paris\nAI: Paris is a fantastic

## Conversation Token Buffer Memory
Similar to the previous one, but this time **you can limit the number of tokens stored in memory**.

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install tiktoken

In [26]:
from langchain.memory import ConversationTokenBufferMemory

In [27]:
token_memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=50
)

/var/folders/r4/zk4dfhln1l78pwrms6p0867h0000gn/T/ipykernel_1997/1318029275.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  token_memory = ConversationTokenBufferMemory(


In [28]:
conversation_token = ConversationChain(
    llm=llm, 
    memory = token_memory,
    verbose=True
)

## Conversation Summary Memory
**Stores a summary** of the previous conversational exchanges.

In [29]:
from langchain.memory import ConversationSummaryBufferMemory

In [30]:
summary_memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100
)

/var/folders/r4/zk4dfhln1l78pwrms6p0867h0000gn/T/ipykernel_1997/4179621716.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryBufferMemory(


In [31]:
conversation_summary = ConversationChain(
    llm=llm, 
    memory = summary_memory,
    verbose=True
)

In [32]:
conversation_summary({"input": """Kurt Cobain dropped out of high school, 
then worked there as a janitor Even though he was by all accounts a slob, 
Kurt Cobain worked as a janitor at Weatherwax High School, not long after 
dropping out of that very school. The dancing janitor in the 
"Smells Like Teen Spirit" music video was an inside joke for 
those who knew of Cobain's old job.
"""})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Kurt Cobain dropped out of high school, 
then worked there as a janitor Even though he was by all accounts a slob, 
Kurt Cobain worked as a janitor at Weatherwax High School, not long after 
dropping out of that very school. The dancing janitor in the 
"Smells Like Teen Spirit" music video was an inside joke for 
those who knew of Cobain's old job.

AI:

> Finished chain.


{'input': 'Kurt Cobain dropped out of high school, \nthen worked there as a janitor Even though he was by all accounts a slob, \nKurt Cobain worked as a janitor at Weatherwax High School, not long after \ndropping out of that very school. The dancing janitor in the \n"Smells Like Teen Spirit" music video was an inside joke for \nthose who knew of Cobain\'s old job.\n',
 'history': '',
 'response': 'That\'s a fascinating piece of trivia! Kurt Cobain\'s time as a janitor at Weatherwax High School is often overlooked in discussions about his early life. It\'s interesting how those details sometimes find their way into his music and videos. The "Smells Like Teen Spirit" music video captured the spirit of youth and rebellion, and incorporating an element like the dancing janitor creates a playful link to Cobain\'s own history. It\'s a reminder of how personal experiences can shape artistic expression. Did you know that Cobain had a strong disdain for mainstream culture, which is reflected i

In [33]:
conversation_summary({"input": """
There were at least five different drummers in the band 
before Dave Grohl. Cobain and Novoselic were always members 
of Nirvana—formerly known as Skid Row, Pen Cap Chew, Bliss, 
and Ted Ed Fred—but finding a permanent drummer proved to be 
even harder than coming up with a decent band name. In the 
beginning, there was trivia answer Aaron Burckhard, who pissed 
off Cobain by getting Kurt's car impounded after being arrested 
for fighting with a police officer. Then there was Melvins 
drummer Dale Crover, who pounded the skins for Cobain and 
Novoselic on their first demo tape before moving to San Francisco. 
Next came Dave Foster, who got arrested for assaulting the 
son of the mayor of Cosmopolis, Washington. Burckhard briefly 
returned before announcing he was too hungover to practice one day. 
Then a mutual friend introduced Cobain and Novoselic to Chad 
Channing, who hung around for two years before the group's 
co-founders decided he wasn't cutting it anymore. Mudhoney 
drummer Dan Peters played on the "Sliver" single.
"""})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human shares that Kurt Cobain dropped out of high school and later worked as a janitor at Weatherwax High School, mentioning that the dancing janitor in the "Smells Like Teen Spirit" music video was an inside joke related to Cobain's past. The AI responds by highlighting the often-overlooked details of Cobain's life and how they influence his music and videos. It notes the playful connection in the music video and reflects on Cobain's disdain for mainstream culture, which contributed to Nirvana's authentic artistic expression. The AI then inquires if the human has a favorite Nirvana song or moment.
Human: 
There were at least five different drummers in the 

{'input': '\nThere were at least five different drummers in the band \nbefore Dave Grohl. Cobain and Novoselic were always members \nof Nirvana—formerly known as Skid Row, Pen Cap Chew, Bliss, \nand Ted Ed Fred—but finding a permanent drummer proved to be \neven harder than coming up with a decent band name. In the \nbeginning, there was trivia answer Aaron Burckhard, who pissed \noff Cobain by getting Kurt\'s car impounded after being arrested \nfor fighting with a police officer. Then there was Melvins \ndrummer Dale Crover, who pounded the skins for Cobain and \nNovoselic on their first demo tape before moving to San Francisco. \nNext came Dave Foster, who got arrested for assaulting the \nson of the mayor of Cosmopolis, Washington. Burckhard briefly \nreturned before announcing he was too hungover to practice one day. \nThen a mutual friend introduced Cobain and Novoselic to Chad \nChanning, who hung around for two years before the group\'s \nco-founders decided he wasn\'t cutting 

In [34]:
conversation_summary({"input": """
Back in Washington, Crover performed with Cobain and Novoselic 
on a seven date tour with Sonic Youth in August 1990, before 
Dave Grohl's band Scream broke up and Melvins frontman Buzz 
Osbourne introduced Grohl to Cobain and Novoselic, ending the 
vicious cycle of rotating drummers.
"""})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human shares that Kurt Cobain dropped out of high school and later worked as a janitor, explaining the inside joke about the dancing janitor in the "Smells Like Teen Spirit" music video. The AI responds by noting the overlooked details of Cobain's life that influenced his art, reflecting on his disdain for mainstream culture and its role in Nirvana's authenticity. The human then discusses the multiple drummers Nirvana had before Dave Grohl, detailing the struggles the band faced in finding a permanent drummer, and mentioning the unique contributions of early drummers like Aaron Burckhard and Dale Crover. The AI comments on how these dynamics informed Nirvan

{'input': "\nBack in Washington, Crover performed with Cobain and Novoselic \non a seven date tour with Sonic Youth in August 1990, before \nDave Grohl's band Scream broke up and Melvins frontman Buzz \nOsbourne introduced Grohl to Cobain and Novoselic, ending the \nvicious cycle of rotating drummers.\n",
 'history': 'System: The human shares that Kurt Cobain dropped out of high school and later worked as a janitor, explaining the inside joke about the dancing janitor in the "Smells Like Teen Spirit" music video. The AI responds by noting the overlooked details of Cobain\'s life that influenced his art, reflecting on his disdain for mainstream culture and its role in Nirvana\'s authenticity. The human then discusses the multiple drummers Nirvana had before Dave Grohl, detailing the struggles the band faced in finding a permanent drummer, and mentioning the unique contributions of early drummers like Aaron Burckhard and Dale Crover. The AI comments on how these dynamics informed Nirvana

In [35]:
print(summary_memory.buffer)

System: The human shares that Kurt Cobain dropped out of high school and worked as a janitor, explaining the inside joke about the dancing janitor in the "Smells Like Teen Spirit" music video. The AI notes overlooked details of Cobain's life that influenced his art and reflects on his disdain for mainstream culture and its role in Nirvana's authenticity. The human discusses Nirvana's multiple drummers before Dave Grohl, detailing struggles in finding a permanent drummer and unique contributions from early drummers like Aaron Burckhard and Dale Crover. The AI comments on how these dynamics informed Nirvana's identity and sound, emphasizing Grohl's pivotal role and asking the human about their favorite stories from Nirvana’s early days. The human mentions Crover's performance with Cobain and Novoselic on a tour with Sonic Youth in 1990 and how Buzz Osbourne introduced Grohl to Nirvana, ending their drummer rotation. The AI highlights Grohl's significant impact on Nirvana's sound and ener

In [36]:
summary_memory.load_memory_variables({})

{'history': 'System: The human shares that Kurt Cobain dropped out of high school and worked as a janitor, explaining the inside joke about the dancing janitor in the "Smells Like Teen Spirit" music video. The AI notes overlooked details of Cobain\'s life that influenced his art and reflects on his disdain for mainstream culture and its role in Nirvana\'s authenticity. The human discusses Nirvana\'s multiple drummers before Dave Grohl, detailing struggles in finding a permanent drummer and unique contributions from early drummers like Aaron Burckhard and Dale Crover. The AI comments on how these dynamics informed Nirvana\'s identity and sound, emphasizing Grohl\'s pivotal role and asking the human about their favorite stories from Nirvana’s early days. The human mentions Crover\'s performance with Cobain and Novoselic on a tour with Sonic Youth in 1990 and how Buzz Osbourne introduced Grohl to Nirvana, ending their drummer rotation. The AI highlights Grohl\'s significant impact on Nirv

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-conversation-buffer.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-conversation-buffer.py